# Regression

---

**Numpy** or **Numerical Python** is the fundamental package for numeric computing with Python. It provides powerful ways to create,
store, and/or manipulate data, which makes it able to seamlessly and speedily integrate with a wide variety
of databases. Numpy is the foundation of several libraries such as `Pandas`, `SciPy`, `SymPy`


In this lecture, we will talk about creating array with certain data types, manipulating array, selecting
elements from arrays, as well as universal functions of NumPy and how to use its statistical and mathematical capabilities. Moreover, we see how to load dataset into array. Such functions are useful for manipulating data and
understanding the functionalities of other common Python data packages.


### Lecture outline

---

* Problem Statement


* Data Description


* EDA - Exploratory Data Analysis


* Data Processing


* Linear Regression


* Decision Tree Regression


* Random Forest Regression


* Model Performance Assessment

#### Reference


[Medical Cost Personal Datasets](https://www.kaggle.com/mirichoi0218/insurance)


[Ordinary Least Squares](https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html)


[sklearn - LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

[sklearn - Decision Tree Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)


[sklearn - Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
# For data processing
import pandas as pd
import numpy as np

# For data viz
import matplotlib.pyplot as plt
import seaborn as sns

# For modeling
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# For model performance assessment
from sklearn import metrics

In [ ]:
plt.style.use("seaborn") # Set plotting style

## Problem Statement

---

To make their profit, insurance companies should collect higher premium than the amount paid to the insured person. Due to this, insurance companies invests a lot of time, effort, and money in creating models that accurately predicts health care costs and then correspondingly charge the insured person.


Today, we build one of the simplest model to make a prediction. However, this simples model will give the best way to interpret the modeling results, compared to other "black-box" machine learning model results.

## Data Description

---

Generally, many factors that affect how much we pay for health insurance are not within our control: diagnosis, type of clinic, city of residence, age and so on. We have no data on the diagnosis of patients. Nonetheless, it's good to have an understanding of what they are. Here are some factors that affect how much health insurance premiums cost for the American people. We will predict individual medical costs billed by health insurance.

$$
$$


* **age**: Age of primary beneficiary


* **sex**: Insurance contractor gender, female or male


* **bmi**: Body Mass Index


* **children**: Number of children covered by health insurance / Number of dependents


* **smoker**: Smoking or not


* **region**: The beneficiary's residential area in the US

In [ ]:
df = pd.read_csv("data/insurance.csv")

In [ ]:
df.head()

## EDA - Exploratory Data Analysis

---

EDA helps us to have a better understanding of our data. In this step we try to extract as much information from our data as possible, such as to guess the data generation process and the distribution of the variables. By having this information we then be able to choose correct model and validate model assumptions. For example, if the data is not normally distributed then we cannot use models which assumed normal distribution in variables and hence we have to change our strategy.

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.shape

### Checking Missing Values

---

We don't have missing values

In [ ]:
df.isnull().sum()

### Univariate Analysis

---

In this stage, we examine variables one-by-one. The best way is to plot the histogram if the variable is continuous or count/frequency plot for categorical variable.

`age` seems to be uniformly distributed

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="age", bins=15);

Histogram of `age` by `smoker` type

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="age", bins=15, hue="smoker", element="step", palette="inferno");

Histogram of `age` by `sex`

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="age", bins=15, hue="sex", element="step", palette="inferno");

`bmi` is normally distributed as expected!

Something strange happens here. The average a=of `bmi` is 30. With a value equal to 30 starts obesity. Are obese people tend to spend more in heath care?

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="bmi", bins=15);

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="bmi", bins=15, hue="sex", element="step", palette="inferno");

This plot indicates that we have more non-smoker in the data then smokers

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="bmi", bins=15, hue="smoker", element="step", palette="inferno");

In [ ]:
df["smoker"].value_counts() # This is the case

The distribution of male and female are the same

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(data=df, x="sex", palette="inferno");

The count plot of `sex` by `smoker`. Plot indicates that there are more male smokers.

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(data=df, x="sex", hue="smoker", palette="inferno");

`smoker` by `region` indicates that in the southeast, there are relatively more smokers

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(data=df, x="region", hue="smoker", palette="inferno");

Histogram of the target variable - `charges`. The distribution of the target variable is right skewed. Later on, we apply log transformation to make it look more normal.

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="charges", bins=25, kde=True);

We can hypothesize that `charges` are more for `smokers` than `non-smokers`.

As we see, `charges` are higher for smokers, indeed.

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="charges", bins=25, kde=True, hue="smoker", element="step", palette="inferno");

Histogram of `charges` by `sex` does not exhibit much difference

In [ ]:
plt.figure(figsize=(10, 8))

sns.histplot(data=df, x="charges", bins=25, kde=True, hue="sex", element="step", palette="inferno");

### Bivariate Analysis

---

In this stage, we analyze the pairs of variables, not single one.

The heatmap of the correlation shows that independent variables are not correlated to the target variable.

In [ ]:
plt.figure(figsize=(8, 6))

sns.heatmap(df.corr(), cmap = "inferno", annot=True);

If we look at the left plot the `charges` the plot is right skewed. In the right plot we will apply natural logarithm and then the plot approximately tends to be normal.

In [ ]:
f= plt.figure(figsize=(18,8))

# For left plot
ax=f.add_subplot(121)

sns.histplot(df["charges"], bins=50, color="red", ax=ax, kde=True)
ax.set_title("Distribution of insurance charges")


# For right plot
ax=f.add_subplot(122)

sns.histplot(np.log(df["charges"]), bins=40, color="blue", ax=ax, kde=True)
ax.set_title("Distribution of insurance charges in $log$ sacle")
ax.set_xscale("log");

Now let's look at the charges by `region`

In [ ]:
charges_by_region = df["charges"].groupby(df["region"]).sum().sort_values(ascending = True)


charges_by_region

In [ ]:
plt.figure(figsize=(10, 8))


sns.barplot(x=charges_by_region, y=charges_by_region.index, palette="inferno");

So, overall the highest medical charges are in the `Southeast` and the lowest are in the `Southwest`. Taking into account certain factors (sex, smoking, having children) let's see how it changes by region.

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot(data=df, x="region", y="charges", hue="sex", palette="inferno");

People who have children, generally smoke less, which the following violinplots shows too.

In [ ]:
plt.figure(figsize=(10, 8))

sns.violinplot(data=df, x="children", y="charges", orient="v", hue="smoker", palette="inferno");

Let now see, how having more children determines the medical charges

In [ ]:
df.groupby("children").agg([np.min, np.max, np.mean])["charges"].round(2)

In the correlation matrix we saw that `charges` column has the highest correlation with `age` and `bmi`. Let see the relationship among these variables.

In [ ]:
plt.figure(figsize=(10, 8))

sns.scatterplot(data=df, x="age", y="charges", palette="inferno");

Scatter plot of `charges` and `age` by `smoker`

In [ ]:
plt.figure(figsize=(10, 8))

sns.scatterplot(data=df, x="age", y="charges", hue="smoker", palette="inferno");

Scatter plot of `charges` and `bmi`

In [ ]:
plt.figure(figsize=(10, 8))

sns.scatterplot(data=df, x="bmi", y="charges", palette="inferno");

Scatter plot of `charges` and `bmi` by `sex`

In [ ]:
plt.figure(figsize=(10, 8))

sns.scatterplot(data=df, x="bmi", y="charges", hue="sex", palette="inferno");

### Visualizing regression models

---

The regression plots in `seaborn` are primarily intended to add a visual guide that helps to emphasize patterns in a dataset during exploratory data analyses. `seaborn` is not itself a package for statistical analysis. To obtain quantitative measures related to the fit of regression models, we should use `statsmodels` and we'll do so. Up until that, exploring a dataset through visualization is just as important as exploring a dataset through tables of statistics.

Before we start modeling part, it's quite interesting to use `seaborn`'s plotting capability to see the best fitted line to only two variables: `age` and `bmi`.

In [ ]:
plt.figure(figsize=(10, 8))

sns.regplot(data=df, x="age", y="charges");

In [ ]:
plt.figure(figsize=(10, 8))

sns.regplot(data=df, x="bmi", y="charges");

## Data Processing

---

In this step, we are processing data in a way to fit the model. Invalid or ill-processed data causes to degrade model performance and hence give biased results.

### Variable Transformation


Machine learning algorithms cannot work with categorical data directly and hence categorical data must be converted to numerical one. Here, I will use two different types of transformation:


* Label Encoding


* One Hot Encoding


$$
$$


**Label Encoding** refers to transforming the alphanumeric labels into numerical form so that the algorithms can understand how to operate on them.


**One hot Encoding** is a representation of categorical variable as binary vectors. In other words, it's a dummy variable or indicator variable.


Note that, when dialing with dummy variables we should not fall in dummy variable trap. The dummy variable trap is a scenario in which the independent variables are multicollinear — a scenario in which two or more variables are highly correlated. In simple terms one variable can be predicted from the others.


Pandas `get_dummies()` method does both label encoding and one hot encoding. However `sklearn` library can do that too.

In [ ]:
categorical_columns = ["sex", "children", "smoker", "region"]

In [ ]:
df = pd.get_dummies(data=df, columns=categorical_columns, drop_first=True)

During the EDA, we saw that out dependent variable `charges` is not normally distributed. Taking log transformation makes it to look more normal. Hence, it's better to convert it into logarithmic scale.

In [ ]:
df["charges"] = np.log(df["charges"])

In [ ]:
df.head()

### Train-Test Split

---

We need to split our data into two parts: train set and test set. On the train set we train the model and on the test set we check the model performance.

In [ ]:
X = df.drop(["charges"], axis = 1)

y = df["charges"]

We have to add a constant term to our `X` dataset. This is a regression constant, sometimes called intercept.

In [ ]:
X.insert(0, "constant", 1)

In [ ]:
X.head()

Now, we are ready to perform split. 80% of data will go for training and 20% for testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=425, shuffle=True)

In [ ]:
print("X train size: \t", x_train.shape)
print("Y train size: \t", y_train.shape)

print("X test size: \t", x_test.shape)
print("Y test size: \t", y_test.shape)

## Linear Regression

---

Linear regression is a supervised learning algorithm used when target / dependent variable is continues real number. It establishes relationship between dependent variable  y  and one or more independent variable  x  using best fit line. In statistics OLS is method to estimate unknown parameter of linear function, it's goal is to minimize sum of square difference between observed dependent variable in the given data set and those predicted by linear regression function.

$$
$$

![alt text](images/lin_reg.jpg "Title")

### Sklearn

In [ ]:
regression_model_1 = LinearRegression() # Create model object

In [ ]:
regression_model_1.fit(x_train, y_train) # Fit model to the data - actual model training

In [ ]:
print("Model Intercept: ", regression_model_1.intercept_)
print()
print("Model Coefficients: ", regression_model_1.coef_)

In [ ]:
prediction_ols = regression_model_1.predict(x_test) # Make prediction on the test set

In [ ]:
print("R Squared: ", metrics.r2_score(y_true=y_test, y_pred=prediction_ols).round(3)) # Calculate goodness of fit

### Statsmodels

In [ ]:
linear_model = sm.OLS(endog=y_train, exog=x_train, hasconst=True) # Create the model object

In [ ]:
result = linear_model.fit() # Fit the model

In [ ]:
result.summary() # Print model results

In [ ]:
prediction = result.predict(x_test) # Make a prediction

### Interpretation of the results

---

When the dependent variable is log-transformed we have to exponentiate the coefficient, subtract one from this number, and multiply by 100. This gives the percent increase (or decrease) in the response for every one-unit increase in the independent variable.

For example: the coefficient for `bmi` is 0.0133. (exp(0.0133) – 1) * 100 = 1.33 For every one-unit increase in the independent variable, our dependent variable increases by about 1.33%


The interpretation of the constant term is that it is an average value. In other words, when all other independent variables are set to zero, the we expect that the `charge` should be (exp(7.0602) - 1) or $1163

## Decision Tree Regression

---

Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

#### Reference


[Decision Trees](https://scikit-learn.org/stable/modules/tree.html#tree)

In [ ]:
tree_regressor = DecisionTreeRegressor(random_state=425) # Create model object

In [ ]:
tree_regressor.fit(x_train, y_train) # Fit the model

In [ ]:
prediction_tree = tree_regressor.predict(x_test)

In [ ]:
print("R Squared: ", metrics.r2_score(y_true=y_test, y_pred=prediction_tree).round(3)) # Calculate goodness of fit

### Feature Importances

---

Decision tree can calculate the feature importance. In other words, feature importance means the share of the contribution to explain dependent variable by independent variables.

In [ ]:
features = x_test.columns.to_list()

importances = tree_regressor.feature_importances_

indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(10, 8))

plt.title("Feature Importances")

plt.barh(range(len(indices)), importances[indices], color="blue", align="center")

plt.yticks(range(len(indices)), [features[i] for i in indices]);

## Random Forest Regression

---

Random Forest represents the ensemble learning algorithm. The goal of ensemble algorithms is to combine the predictions of several weak estimators, such as decision tree in order to improve model performance. Particularly, random forest as name suggests is the forest of the decision trees.

In [ ]:
forest_regressor = RandomForestRegressor(n_estimators=150, max_depth=2, random_state=425) # Create model object

In [ ]:
forest_regressor.fit(x_train, y_train) # Fit the model

In [ ]:
prediction_forest = forest_regressor.predict(x_test)

In [ ]:
print("R Squared: ", metrics.r2_score(y_true=y_test, y_pred=prediction_forest).round(3)) # Calculate goodness of fit

### Feature Importance

In [ ]:
features = x_test.columns.to_list()

importances = forest_regressor.feature_importances_

indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(10, 8))

plt.title("Feature Importances")

plt.barh(range(len(indices)), importances[indices], color="blue", align="center")

plt.yticks(range(len(indices)), [features[i] for i in indices]);

## Model Performance Assessment

---

There are 3 main metrics for model evaluation in regression:

1. R Square and Adjusted R Square


2. Mean Square Error(MSE) and Root Mean Square Error(RMSE)


3. Mean Absolute Error(MAE)

### R Squared and Adjusted R Squared

---

Adjusted R Squared penalizes R Squared not to be monotonically increasing.

![alt text](images/r2.png "Title")

![alt text](images/adjr2.png "Title")

$$
$$

#### Where, $N$ is the number of samples (observations) and $p$ is the number of predictors (independent variables)

In [ ]:
print("R Squared - OLS: ", metrics.r2_score(y_true=y_test, y_pred=prediction_ols).round(3))

print("R Squared - Tree: ", metrics.r2_score(y_true=y_test, y_pred=prediction_tree).round(3))

print("R Squared - Forest: ", metrics.r2_score(y_true=y_test, y_pred=prediction_forest).round(3))

### Mean Square Error(MSE) and Root Mean Square Error(RMSE)

---

`MSE` measures the average of the squares of the errors. That is, the average squared difference between the estimated values and the actual values.

![alt text](images/mse.png "Title")

![alt text](images/rmse.png "Title")

In [ ]:
print("MSE - OLS: ", metrics.mean_squared_error(y_true=y_test, y_pred=prediction_ols).round(3))

print("MSE - Tree: ", metrics.mean_squared_error(y_true=y_test, y_pred=prediction_tree).round(3))

print("MSE - Forest: ", metrics.mean_squared_error(y_true=y_test, y_pred=prediction_forest).round(3))

### Mean Absolute Error(MAE)

---

It's an arithmetic average of absolute error in our measurement

![alt text](images/mae1.png "Title")

In [ ]:
print("MAE - OLS: ", metrics.mean_absolute_error(y_true=y_test, y_pred=prediction_ols).round(3))

print("MAE - Tree: ", metrics.mean_absolute_error(y_true=y_test, y_pred=prediction_tree).round(3))

print("MAE - Forest: ", metrics.mean_absolute_error(y_true=y_test, y_pred=prediction_forest).round(3))

# Summary

---

In this lecture, we've covered the most exciting part of a data science. In this stage, we try to predict the future or to find out the relation between dependent and independent variables. However, the models we saw today, are rather simple, yet the most interpretable models out there in ML world.


Always remember, we have a trade off. On the one hand we have a complex model which predicts the future more accurately but fails to be interpretable and on the second hand we have simple models, which are less accurate but is we can interpret their result easily.


**Make wise decision. Complex models are not always necessary!!!**